In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [60]:
page_number = 1
initial_url = 'https://jobinja.ir/jobs?&b=&filters%5Bjob_categories%5D%5B0%5D=&filters%5Bkeywords%5D%5B0%5D=&filters%5Blocations%5D%5B0%5D='

# Create an empty list to store job data dictionaries
job_data_list = []

for i in range(1,10):
    url = initial_url + f'&page={page_number}'
    response = requests.get(url)

    if response.status_code != 200:
        break  # Stop crawling if the page is not accessible or doesn't exist

    soup = BeautifulSoup(response.content, 'html.parser')
    job_links = soup.find_all('a', class_='c-jobListView__titleLink')

    if not job_links:
        break  # Stop crawling if there are no job links on the page

    for job_link in job_links:
        job_url = job_link['href']
        job_response = requests.get(job_url)

        if job_response.status_code == 200:
            job_soup = BeautifulSoup(job_response.content, 'html.parser')
            company_name_element = job_soup.find('h2', class_='c-companyHeader__name')
    
            # Get the company name (both Persian and English)
            company_name = company_name_element.text.strip().split('|')[0].strip()
    
            # Find the company meta information
            company_meta_elements = job_soup.find_all('span', class_='c-companyHeader__metaItem')
            
            # i wish whoever came up with this name tagging idea for the website a painful death i had
            # to start over two times after crawling through the entire website, scraping 26k data because
            # around 2.5k fields (around 10% of data) in company category/size had their values switched up...
            if len(company_meta_elements) > 3:
                print(len(company_meta_elements))
                
                company_category = company_meta_elements[1].find('a', class_='c-companyHeader__metaLink').text.strip()
                company_size = company_meta_elements[2].text.strip()
                company_website = company_meta_elements[3].find('a', class_='c-companyHeader__metaLink').text.strip()
            else:
                # Get the company category (only the Persian phrase)
                company_category_element = company_meta_elements[0].find('a', class_='c-companyHeader__metaLink')
                company_category = company_category_element.text.strip() if company_category_element else ''
        
                # Get the company size (only the Persian phrase)
                company_size = company_meta_elements[1].text.strip() if len(company_meta_elements) > 1 else ''
        
                # Get the company website (if available)
                company_website_element = company_meta_elements[-1].find('a', class_='c-companyHeader__metaLink', target='_blank')
                company_website = company_website_element['href'].replace('https://', '').replace('http://', '').replace('/', '') if company_website_element else ''
                
            
    
            # Get the job position
            job_position_element = job_soup.find('div', class_='c-jobView__title').find('h1')
            job_position = job_position_element.text.strip() if job_position_element else ''
            
            # Pulling all 5 elements from the job info box
            info_box_element = job_soup.find('ul', class_='c-jobView__firstInfoBox c-infoBox').find_all('li', class_='c-infoBox__item')
            
            # Job category
            job_category = info_box_element[0].find('span', class_='black').text.strip()
            
            # Job location
            job_location = info_box_element[1].find('span', class_='black').text.strip().split('،')[0].strip()
            
            # Job employment type
            job_employment_type = info_box_element[2].find('span', class_='black').text.strip()
            
            # Job least experience needed
            job_experience = info_box_element[3].find('span', class_='black').text.strip()
            
            # Job salary
            job_salary = info_box_element[4].find('span', class_='black').text.strip()
            
            
            # Find the skills box
            skills_box_element = job_soup.find('ul', class_='c-infoBox u-mB0')
            skills_li_element = skills_box_element.find('h4', text='مهارت‌های مورد نیاز').parent
            # Get all the skill tags with class "black" within the skills box
            skill_tags = skills_li_element.find_all('span', class_='black')
            # Extract the skills and join them with a comma
            skills = ', '.join(skill_tag.text.strip() for skill_tag in skill_tags)
    
            # Find the gender box
            try:
                gender_li_element = skills_box_element.find('h4', text='جنسیت').parent
                gender = gender_li_element.find('span', class_='black').text.strip()
            except AttributeError:
                gender = ''
    
            # Find the military service box (is None when gender is set to 'female')
            try:
                military_service_li_element = skills_box_element.find('h4', text='وضعیت نظام وظیفه').parent
                military_service = military_service_li_element.find('span', class_='black').text.strip()
            except AttributeError:
                military_service = ''
    
            # Find the education box
            try:
                education_li_element = skills_box_element.find('h4', text='حداقل مدرک تحصیلی').parent
                education = education_li_element.find('span', class_='black').text.strip()
            except AttributeError:
                education = ''
    
            # Find the job description
            job_description_element = job_soup.find('div', class_='o-box__text s-jobDesc c-pr40p')
            job_description = job_description_element.get_text(strip=True) if job_description_element else ''
            
            
            # Append the job data to the list
            job_data_list.append({
                'Job Position': job_position,
                'Job Category': job_category,
                'Job Location': job_location,
                'Employment Type': job_employment_type,
                'Experience': job_experience,
                'Salary': job_salary,
                'Company Name': company_name,
                'Company Category': company_category,
                'Company Size': company_size,
                'Company Website': company_website,
                'Skills': skills,
                'Gender': gender,
                'Military Service': military_service,
                'Education': education,
                'Job Description': job_description,
                'Job URL': job_url
            })
    
            # Checks
            print(company_name)
            print(company_category)
            print(company_size)
            print(company_website)
            print(job_position)
            print(job_category)
            print(job_location)
            print(job_employment_type)
            print(job_experience)
            print(job_salary)
            print(skills)
            print(gender)
            print(military_service)
            print(education)
            print(job_description)
            print(job_url)
            print('-' * 50)

# Create a DataFrame from the collected job data list
df = pd.DataFrame(job_data_list)

# Print the DataFrame with the collected job specifications
print(df)

C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1320484428.py:82: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  skills_li_element = skills_box_element.find('h4', text='مهارت‌های مورد نیاز').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1320484428.py:90: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gender_li_element = skills_box_element.find('h4', text='جنسیت').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1320484428.py:97: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  military_service_li_element = skills_box_element.find('h4', text='وضعیت نظام وظیفه').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1320484428.py:104: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  education_li_element = skills_box_element.find('h4', text='حداقل مدرک تحصیل

وب متریک
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر
webmetric.ir
استخدام برنامه‌نویس iOS
وب،‌ برنامه‌نویسی و نرم‌افزار
تهران
تمام وقت
سه تا شش سال
از ۳۰,۰۰۰,۰۰۰ تومان
IOS, Git, Swift, Objective-C
مهم نیست
مهم‌ نیست
مهم نیست

https://jobinja.ir/companies/webmetric/jobs/Cvhe/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-ios-%D8%AF%D8%B1-%D9%88%D8%A8-%D9%85%D8%AA%D8%B1%DB%8C%DA%A9?_ref=16
--------------------------------------------------
بازرگانی پیشتازان سگال مهر
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر

استخدام حسابدار (خانم)
مالی و حسابداری
تهران
تمام وقت
کمتر از سه سال
توافقی
مالی و حسابداری, امور مالی, امور مالیاتی
زن

مهم نیست
شرکت بازرگانی پیشتازان سگال مهربه منظور تکمیل کادر خود از واجدین شرایط ذیل دعوت به همکاری می نماید :-مهارت ها :- آشنا به اصول حسابداری- آشنا به ثبت اسناد حسابداری- ثبت خرید ها- ثبت دریافت ها و پرداخت ها- مغایرت گیری بانک ها- سابقه کار با نرم افزارهای مربوطه- داشتن انگیزه بلند مدت در کار- م

سپیدماکیان
کالای مصرفی و خوراکی
بیش از ۱۰۰۰ نفر
sepidmakian.com
استخدام کارشناس فروش تلفنی (خانم)
فروش و بازاریابی
تهران
تمام وقت
مهم نیست
از ۱۲,۰۰۰,۰۰۰ تومان
فروش تلفنی, اصول و فنون مذاکره
زن

کارشناسی
اگر به دنبال محیط پویا و شاداب کاری هستید و خود را فردی پر انرژی، مسئولیت پذیر و خلاق با روحیه پیگیری مستمر می دانید، ما در سپیدماکیان منتظر شما هستیم.ثبت روزانه سفارش و کلیه امور مربوط به مشتریانساعت کاری: شنبه تا چهارشنبه 7:30 الی 15:50 و پنجشنبه ها 7:30 الی 12:20محدوده: فرحزادی، اتوبان یادگار امام شمال
https://jobinja.ir/companies/sepid-makian-1/jobs/CmQt/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%DA%A9%D8%A7%D8%B1%D8%B4%D9%86%D8%A7%D8%B3-%D9%81%D8%B1%D9%88%D8%B4-%D8%AA%D9%84%D9%81%D9%86%DB%8C-%D8%AE%D8%A7%D9%86%D9%85-%D8%AF%D8%B1-%D8%B3%D9%BE%DB%8C%D8%AF%D9%85%D8%A7%DA%A9%DB%8C%D8%A7%D9%86?_ref=16
--------------------------------------------------
نوید درمان پارس
خدمات درمانی، پزشکی و سلامت
۲ - ۱۰ نفر

استخدام کمک حسابدار
مالی و حسابداری
تهران
تمام وقت
کمتر از سه سال
از ۸,۰۰۰,۰۰۰ ت

4
پارس پک
کامپیوتر، فناوری اطلاعات و اینترنت
۵۱ تا ۲۰۰ نفر
parspack.com
استخدام کارشناس پشتیبانی امور مشتریان(حوزه هاستینگ-دورکاری)
پشتیبانی و امور مشتریان
تهران
تمام وقت
کمتر از سه سال
توافقی
ارتباط با مشتریان و پشتیبانی, پشتیبانی, پشتیبانی هاستینگ
مهم نیست
مهم‌ نیست
کارشناسی
شرکت پارس پک مشتاق به جذب همکاران پرتلاش برای تیم پشتیبانی خود است. اگر فکر می‌کنید به رشد در این مسیر شغلی علاقه‌ مندید و روحیه مناسب این موقعیت شغلی رو دارید، خیلی خوشحال می شویم شما را در پارس پک ببینیم.وظایف و مسئولیت ها:پاسخگویی به تماس های ورودی و  خروجی ،درج اطلاعات و تگ ها در CRMپاسخگویی به تیکت های فنیپاسخگویی به چت آنلاین در صورت نیازراهنمایی کامل دسترسی به پنل سرویس هابررسی ثبت سفارش کاربر و خطاهای احتمالیپیگیری درخواست کاربرانبررسی گزارش های ماهانهمهارت ها و ویژگی های مورد نیاز:مسلط به وردپرس  آشنا به فایل ها، تنظیمات و خطا های آنآشنا با کنترل پنل های هاستینگآشنایی با مفاهیم اولیه شبکه (ip-port و سرویس ها داخل شبکه)مهارت بالا در برقراری ارتباط با کاربرانآشنا با خدمات هاستینگ و نحوه کار با  سرویس های ا

دایان تجارت کارن
واردات و صادرات
۲ - ۱۰ نفر

استخدام کارشناس فروش تلفنی
فروش و بازاریابی
تهران
تمام وقت
کمتر از سه سال
توافقی
فروش تلفنی, اصول و فنون مذاکره, فروش و بازاریابی
مهم نیست
معافیت دائم
کارشناسی
دارای روابط عمومی بالابا انگیزه و دارای پشتکارمهارت در مذاکره و متقاعد سازیمهارت فروش تلفنیشخصیت منظم و پیگیر●پیش‌نیازهاتوانایی انجام کار گروهی.داشتن روحیه خلاق.مسلط به مایکروسافت آفیس.حضور به موقع و مرتب و منظم و حفظ نظم و انضباط عمومی در محیط کارکمک به توسعه و انتقال مهارتها و آموزش های احراز شده به سایر پرسنل مرتبط
https://jobinja.ir/companies/dayan-tejarat-karen/jobs/CkIr/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%DA%A9%D8%A7%D8%B1%D8%B4%D9%86%D8%A7%D8%B3-%D9%81%D8%B1%D9%88%D8%B4-%D8%AA%D9%84%D9%81%D9%86%DB%8C-%D8%AF%D8%B1-%D8%AF%D8%A7%DB%8C%D8%A7%D9%86-%D8%AA%D8%AC%D8%A7%D8%B1%D8%AA-%DA%A9%D8%A7%D8%B1%D9%86?_ref=16
--------------------------------------------------
رسمیو
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر
rasm.io
استخدام کارشناس فروش
فروش و بازاریابی
تهران
تمام وقت


...

دایان تجارت کارن
واردات و صادرات
۲ - ۱۰ نفر

استخدام کارشناس فروش تلفنی
فروش و بازاریابی
تهران
تمام وقت
کمتر از سه سال
توافقی
فروش تلفنی, اصول و فنون مذاکره, فروش و بازاریابی
مهم نیست
معافیت دائم
کارشناسی
دارای روابط عمومی بالابا انگیزه و دارای پشتکارمهارت در مذاکره و متقاعد سازیمهارت فروش تلفنیشخصیت منظم و پیگیر●پیش‌نیازهاتوانایی انجام کار گروهی.داشتن روحیه خلاق.مسلط به مایکروسافت آفیس.حضور به موقع و مرتب و منظم و حفظ نظم و انضباط عمومی در محیط کارکمک به توسعه و انتقال مهارتها و آموزش های احراز شده به سایر پرسنل مرتبط
https://jobinja.ir/companies/dayan-tejarat-karen/jobs/CkIr/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%DA%A9%D8%A7%D8%B1%D8%B4%D9%86%D8%A7%D8%B3-%D9%81%D8%B1%D9%88%D8%B4-%D8%AA%D9%84%D9%81%D9%86%DB%8C-%D8%AF%D8%B1-%D8%AF%D8%A7%DB%8C%D8%A7%D9%86-%D8%AA%D8%AC%D8%A7%D8%B1%D8%AA-%DA%A9%D8%A7%D8%B1%D9%86?_ref=16
--------------------------------------------------
رسمیو
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر
rasm.io
استخدام کارشناس فروش
فروش و بازاریابی
تهران
تمام وقت


اوژن مشرق پارس
کامپیوتر، فناوری اطلاعات و اینترنت
۵۱ تا ۲۰۰ نفر
www.ompfinex.com
استخدام SRE) Site Reliability Engineer)
IT / DevOps / Server
تهران
تمام وقت
مهم نیست
توافقی
SRE, CI/CD, DevOps
مهم نیست
مهم‌ نیست
مهم نیست

https://jobinja.ir/companies/omp-finex/jobs/Cmfv/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-sre-site-reliability-engineer-%D8%AF%D8%B1-%D8%A7%D9%88%DA%98%D9%86-%D9%85%D8%B4%D8%B1%D9%82-%D9%BE%D8%A7%D8%B1%D8%B3?_ref=16
--------------------------------------------------
مشاطب
خدمات درمانی، پزشکی و سلامت
۲۰۱ تا ۵۰۰ نفر
www.mashateb.com
استخدام منشی و مسئول دفتر (آقا)
مسئول دفتر، اجرائی و اداری
تهران
تمام وقت
سه تا شش سال
از ۱۰,۰۰۰,۰۰۰ تومان
پاسخگویی به تلفن, پیگیری امور, امور اداری
مرد
معافیت دائم
کاردانی
- مدرک تحصیلی کاردانی کامپیوتر- دارای حداقل سه سال سابقه کار در امور مسئول دفتر- تسلط بر نرم افزار افیس- دارای روابط عمومی خوب و ظاهری آراسته- علاقمند به کار- منظم و دقیق
https://jobinja.ir/companies/mashateb-1/jobs/CGjN/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%85%

وب متریک
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر
webmetric.ir
استخدام برنامه‌نویس iOS
وب،‌ برنامه‌نویسی و نرم‌افزار
تهران
تمام وقت
سه تا شش سال
از ۳۰,۰۰۰,۰۰۰ تومان
IOS, Git, Swift, Objective-C
مهم نیست
مهم‌ نیست
مهم نیست

https://jobinja.ir/companies/webmetric/jobs/Cvhe/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-ios-%D8%AF%D8%B1-%D9%88%D8%A8-%D9%85%D8%AA%D8%B1%DB%8C%DA%A9?_ref=16
--------------------------------------------------
بازرگانی پیشتازان سگال مهر
کامپیوتر، فناوری اطلاعات و اینترنت
۱۱ تا ۵۰ نفر

استخدام حسابدار (خانم)
مالی و حسابداری
تهران
تمام وقت
کمتر از سه سال
توافقی
مالی و حسابداری, امور مالی, امور مالیاتی
زن

مهم نیست
شرکت بازرگانی پیشتازان سگال مهربه منظور تکمیل کادر خود از واجدین شرایط ذیل دعوت به همکاری می نماید :-مهارت ها :- آشنا به اصول حسابداری- آشنا به ثبت اسناد حسابداری- ثبت خرید ها- ثبت دریافت ها و پرداخت ها- مغایرت گیری بانک ها- سابقه کار با نرم افزارهای مربوطه- داشتن انگیزه بلند مدت در کار- م

سپیدماکیان
کالای مصرفی و خوراکی
بیش از ۱۰۰۰ نفر
sepidmakian.com
استخدام کارشناس فروش تلفنی (خانم)
فروش و بازاریابی
تهران
تمام وقت
مهم نیست
از ۱۲,۰۰۰,۰۰۰ تومان
فروش تلفنی, اصول و فنون مذاکره
زن

کارشناسی
اگر به دنبال محیط پویا و شاداب کاری هستید و خود را فردی پر انرژی، مسئولیت پذیر و خلاق با روحیه پیگیری مستمر می دانید، ما در سپیدماکیان منتظر شما هستیم.ثبت روزانه سفارش و کلیه امور مربوط به مشتریانساعت کاری: شنبه تا چهارشنبه 7:30 الی 15:50 و پنجشنبه ها 7:30 الی 12:20محدوده: فرحزادی، اتوبان یادگار امام شمال
https://jobinja.ir/companies/sepid-makian-1/jobs/CmQt/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%DA%A9%D8%A7%D8%B1%D8%B4%D9%86%D8%A7%D8%B3-%D9%81%D8%B1%D9%88%D8%B4-%D8%AA%D9%84%D9%81%D9%86%DB%8C-%D8%AE%D8%A7%D9%86%D9%85-%D8%AF%D8%B1-%D8%B3%D9%BE%DB%8C%D8%AF%D9%85%D8%A7%DA%A9%DB%8C%D8%A7%D9%86?_ref=16
--------------------------------------------------
نوید درمان پارس
خدمات درمانی، پزشکی و سلامت
۲ - ۱۰ نفر

استخدام کمک حسابدار
مالی و حسابداری
تهران
تمام وقت
کمتر از سه سال
از ۸,۰۰۰,۰۰۰ ت

4
پارس پک
کامپیوتر، فناوری اطلاعات و اینترنت
۵۱ تا ۲۰۰ نفر
parspack.com
استخدام کارشناس پشتیبانی امور مشتریان(حوزه هاستینگ-دورکاری)
پشتیبانی و امور مشتریان
تهران
تمام وقت
کمتر از سه سال
توافقی
ارتباط با مشتریان و پشتیبانی, پشتیبانی, پشتیبانی هاستینگ
مهم نیست
مهم‌ نیست
کارشناسی
شرکت پارس پک مشتاق به جذب همکاران پرتلاش برای تیم پشتیبانی خود است. اگر فکر می‌کنید به رشد در این مسیر شغلی علاقه‌ مندید و روحیه مناسب این موقعیت شغلی رو دارید، خیلی خوشحال می شویم شما را در پارس پک ببینیم.وظایف و مسئولیت ها:پاسخگویی به تماس های ورودی و  خروجی ،درج اطلاعات و تگ ها در CRMپاسخگویی به تیکت های فنیپاسخگویی به چت آنلاین در صورت نیازراهنمایی کامل دسترسی به پنل سرویس هابررسی ثبت سفارش کاربر و خطاهای احتمالیپیگیری درخواست کاربرانبررسی گزارش های ماهانهمهارت ها و ویژگی های مورد نیاز:مسلط به وردپرس  آشنا به فایل ها، تنظیمات و خطا های آنآشنا با کنترل پنل های هاستینگآشنایی با مفاهیم اولیه شبکه (ip-port و سرویس ها داخل شبکه)مهارت بالا در برقراری ارتباط با کاربرانآشنا با خدمات هاستینگ و نحوه کار با  سرویس های ا

In [17]:
def extract_job_data(job_soup):
    # Get the company name (both Persian and English)
    company_name_element = job_soup.find('h2', class_='c-companyHeader__name')
    company_name = company_name_element.text.strip().split('|')[0].strip()

    # Find the company meta information
    company_meta_elements = job_soup.find_all('span', class_='c-companyHeader__metaItem')

    # Get the company category (only the Persian phrase)
    company_category_element = company_meta_elements[0].find('a', class_='c-companyHeader__metaLink')
    company_category = company_category_element.text.strip() if company_category_element else ''

    # Get the company size (only the Persian phrase)
    company_size = company_meta_elements[1].text.strip() if len(company_meta_elements) > 1 else ''

    # Get the company website (if available)
    company_website_element = company_meta_elements[-1].find('a', class_='c-companyHeader__metaLink', target='_blank')
    company_website = company_website_element['href'].replace('https://', '').replace('http://', '').replace('/', '') if company_website_element else ''

    # Get the job position
    job_position_element = job_soup.find('div', class_='c-jobView__title').find('h1')
    job_position = job_position_element.text.strip() if job_position_element else ''

    # Find the skills box
    skills_box_element = job_soup.find('ul', class_='c-infoBox u-mB0')
    skills_li_element = skills_box_element.find('h4', text='مهارت‌های مورد نیاز').parent
    # Get all the skill tags with class "black" within the skills box
    skill_tags = skills_li_element.find_all('span', class_='black')
    # Extract the skills and join them with a comma
    skills = ', '.join(skill_tag.text.strip() for skill_tag in skill_tags)

    # Find the gender box
    try:
        gender_li_element = skills_box_element.find('h4', text='جنسیت').parent
        gender = gender_li_element.find('span', class_='black').text.strip()
    except AttributeError:
        gender = ''

    # Find the military service box (is None when gender is set to 'female')
    try:
        military_service_li_element = skills_box_element.find('h4', text='وضعیت نظام وظیفه').parent
        military_service = military_service_li_element.find('span', class_='black').text.strip()
    except AttributeError:
        military_service = ''

    # Find the education box
    try:
        education_li_element = skills_box_element.find('h4', text='حداقل مدرک تحصیلی').parent
        education = education_li_element.find('span', class_='black').text.strip()
    except AttributeError:
        education = ''

    # Find the job description
    job_description_element = job_soup.find('div', class_='o-box__text s-jobDesc c-pr40p')
    job_description = job_description_element.get_text(strip=True) if job_description_element else ''
    
    return {
        'Job Category': job_category,
        'Job Position': job_position,
        'Company Name': company_name,
        'Company Category': company_category,
        'Company Size': company_size,
        'Company Website': company_website,
        'Skills': skills,
        'Gender': gender,
        'Military Service': military_service,
        'Education': education,
        'Job Description': job_description
    }

In [18]:
def crawl_job_data():
    initial_url = 'https://jobinja.ir/jobs?&b=&filters%5Bjob_categories%5D%5B0%5D=&filters%5Bkeywords%5D%5B0%5D=&filters%5Blocations%5D%5B0%5D='

    # Create an empty list to store job data dictionaries
    job_data_list = []
    page_number = 1

    while page_number <= 5:
        url = initial_url + f'&page={page_number}'
        response = requests.get(url)

        if response.status_code != 200:
            break  # Stop crawling if the page is not accessible or doesn't exist

        soup = BeautifulSoup(response.content, 'html.parser')
        job_links = soup.find_all('a', class_='c-jobListView__titleLink')

        if not job_links:
            break  # Stop crawling if there are no job links on the page

        for entry, job_link in enumerate(job_links, 1):
            job_url = job_link['href']
            job_response = requests.get(job_url)

            if job_response.status_code == 200:
                job_soup = BeautifulSoup(job_response.content, 'html.parser')
                job_data = extract_job_data(job_soup)
                job_data['Job URL'] = job_url

                # Append the job data to the list
                job_data_list.append(job_data)

                # Print processing information for each job entry
                print(f"Processing Page: {page_number} - Entry: {entry}")

        page_number += 1
        print(f'20 Entries from {page_number} appended to the DataFrame')


    return job_data_list

In [19]:
page_number = 1
job_data_list = crawl_job_data()

# Create a DataFrame from the collected job data list
df = pd.DataFrame(job_data_list)

# Print the DataFrame with the collected job specifications
print(df)

C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1272749541.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  skills_li_element = skills_box_element.find('h4', text='مهارت‌های مورد نیاز').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1272749541.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gender_li_element = skills_box_element.find('h4', text='جنسیت').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1272749541.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  military_service_li_element = skills_box_element.find('h4', text='وضعیت نظام وظیفه').parent
C:\Users\Hexa\AppData\Local\Temp\ipykernel_17272\1272749541.py:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  education_li_element = skills_box_element.find('h4', text='حداقل مدرک تحصیلی

Processing Page: 1 - Entry: 1
Processing Page: 1 - Entry: 2
Processing Page: 1 - Entry: 3
Processing Page: 1 - Entry: 4
Processing Page: 1 - Entry: 5
Processing Page: 1 - Entry: 6
Processing Page: 1 - Entry: 7
Processing Page: 1 - Entry: 8
Processing Page: 1 - Entry: 9
Processing Page: 1 - Entry: 10
Processing Page: 1 - Entry: 11
Processing Page: 1 - Entry: 12
Processing Page: 1 - Entry: 13
Processing Page: 1 - Entry: 14
Processing Page: 1 - Entry: 15
Processing Page: 1 - Entry: 16
Processing Page: 1 - Entry: 17
Processing Page: 1 - Entry: 18
Processing Page: 1 - Entry: 19
Processing Page: 1 - Entry: 20
20 Entries from 2 appended to the DataFrame
Processing Page: 2 - Entry: 1
Processing Page: 2 - Entry: 2
Processing Page: 2 - Entry: 3
Processing Page: 2 - Entry: 4
Processing Page: 2 - Entry: 5
Processing Page: 2 - Entry: 6
Processing Page: 2 - Entry: 7
Processing Page: 2 - Entry: 8
Processing Page: 2 - Entry: 9
Processing Page: 2 - Entry: 10
Processing Page: 2 - Entry: 11
Processing Pa

In [61]:
df.head()

,Job Position,Job Category,Job Location,Employment Type,Experience,Salary,Company Name,Company Category,Company Size,Company Website,Skills,Gender,Military Service,Education,Job Description,Job URL
0,استخدام برنامه‌نویس iOS,وب،‌ برنامه‌نویسی و نرم‌افزار,تهران,تمام وقت,سه تا شش سال,"از ۳۰,۰۰۰,۰۰۰ تومان",وب متریک,کامپیوتر، فناوری اطلاعات و اینترنت,۱۱ تا ۵۰ نفر,webmetric.ir,"IOS, Git, Swift, Objective-C",مهم نیست,مهم‌ نیست,مهم نیست,,https://jobinja.ir/companies/webmetric/jobs/Cv...
1,استخدام حسابدار (خانم),مالی و حسابداری,تهران,تمام وقت,کمتر از سه سال,توافقی,بازرگانی پیشتازان سگال مهر,کامپیوتر، فناوری اطلاعات و اینترنت,۱۱ تا ۵۰ نفر,,"مالی و حسابداری, امور مالی, امور مالیاتی",زن,,مهم نیست,شرکت بازرگانی پیشتازان سگال مهربه منظور تکمیل ...,https://jobinja.ir/companies/pishtazan-segal-m...
2,استخدام مونتاژ کار الکترونیک (آقا),مهندسی برق و الکترونیک,تهران,تمام وقت,سه تا شش سال,توافقی,مینفری,معماری و عمران,۲ - ۱۰ نفر,www.minferi.ir,"قطعات الکترونیکی و مونتاژ, مونتاژ, امور فنی",مرد,مهم‌ نیست,دیپلم,شرکت مهندسی مجریان رعد پرشین در تهران (کهریزک ...,https://jobinja.ir/companies/minferi/jobs/CSbJ...
3,استخدام کارشناس حسابداری,مالی و حسابداری,تهران,تمام وقت,کمتر از سه سال,توافقی,دایان تجارت کارن,واردات و صادرات,۲ - ۱۰ نفر,,"امور مالی, مالی و حسابداری, حسابداری",مهم نیست,معافیت دائم,کارشناسی,انجام امور قانونی و اقدامات قبلی جهت ارسال گزا...,https://jobinja.ir/companies/dayan-tejarat-kar...
4,استخدام کارشناس حسابداری,مالی و حسابداری,تهران,تمام وقت,کمتر از سه سال,"از ۱۲,۰۰۰,۰۰۰ تومان",آژانس تبلیغات دیجیتال دیما,تبلیغات، بازاریابی و برندسازی,۵۱ تا ۲۰۰ نفر,deema.agency,"مالی و حسابداری, امور مالی, امور مالیاتی",مهم نیست,مهم‌ نیست,کارشناسی,حداقل ۱ سال سابقه عملیاتیمسئولیت ها:حسابداری ف...,https://jobinja.ir/companies/deema-advertising...


In [62]:
df['Employment Type']

0      تمام وقت
1      تمام وقت
2      تمام وقت
3      تمام وقت
4      تمام وقت
         ...   
175    تمام وقت
176    تمام وقت
177    تمام وقت
178    تمام وقت
179    تمام وقت
Name: Employment Type, Length: 180, dtype: object

In [63]:
df.to_csv('Jobinja.csv', index=False, encoding='utf-8-sig')# Print the DataFrame with the collected job specifications